In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

#from sklearn import preprocessing as pp
#from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import re
import time
from tqdm import tqdm
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

import nltk
from bs4 import BeautifulSoup
import transformers
from transformers import AdamW

#import torchvision.transforms as transforms
import torch.optim as optimizers
from torch.utils.data import random_split
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import WeightedRandomSampler, BatchSampler

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#set random seed
def set_seed(seed: int = 123):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
set_seed(1234)

In [ ]:
class CFWholeDataset(Dataset):
    def __init__(self, csv_file, tokenizer=None):
        self.csv_file = csv_file
        self.transform = transform
        self.tokenizer = tokenizer
        self.feature_cols = FEATURE_COLS
        self.target_cols = TARGET_COLS
        
        
    def __len__(self):
        return len(self.csv_file)
    
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        features = self.csv_file[self.feature_cols].iloc[idx]
        features = torch.FloatTensor(features)    
        html_text = self.csv_file.html_raw.iloc[idx]
        if self.tokenizer:
            html_text = self.tokenizer(html_text)
            
        try:
            target = self.csv_file[self.target_cols].iloc[idx]
            target = torch.tensor(target)
            # train_step
        except:
            target = self.csv_file["id"].iloc[idx]
            # test_step
        
        if self.transform:
            features = self.transform(features)
            
        return features, html_text, target

In [5]:
#define transform
class BERT_Tokenize(object):
    def __init__(self, model_type, max_len):
        self.max_len = max_len
        
        if model_type == "BERT" or model_type == "TAPTBERT":
            from transformers import BertTokenizer, BertForSequenceClassification
            self.bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
            
        elif model_type == "ALBERT":
            from transformers import AlbertTokenizer, AlbertForSequenceClassification
            self.bert_tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")
            
        elif model_type == "XLNET":
            from transformers import XLNetTokenizer, XLNetForSequenceClassification
            self.bert_tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
        
        elif model_type == "ROBERTA":
            from transformers import RobertaTokenizer, RobertaForSequenceClassification
            self.bert_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
        
        elif model_type == "csROBERTA":
            from transformers import AutoTokenizer, AutoModel
            self.bert_tokenizer = AutoTokenizer.from_pretrained("allenai/cs_roberta_base")
            
        elif model_type == "XLMROBERTA":
            from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
            self.bert_tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
            
        elif model_type == "ELECTRA":
            from transformers import ElectraTokenizer, ElectraForSequenceClassification
            self.bert_tokenizer = ElectraTokenizer.from_pretrained("google/electra-base-discriminator")
            
    
    def __call__(self,text):
        inputs = self.bert_tokenizer.encode_plus(
                        text,                       # Sentence to encode.
                        add_special_tokens = True,  # Add '[CLS]' and '[SEP]'
                        max_length = self.max_len,  # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,  # Construct attn. masks.
                   )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        del text, inputs
        return np.array(ids), np.array(mask)

In [ ]:
class BertHead(nn.Module):
    def __init__(self, model_type, num_classes):
        super(BertHead, self).__init__()
        self.model_type = True if model_type == "TAPTBERT" else False
        
        if model_type == "ALBERT":
            from transformers import AlbertTokenizer, AlbertForSequenceClassification
            self.base_model = AlbertForSequenceClassification.from_pretrained(
                "albert-base-v2",early_stopping=False,num_labels=num_classes)
            
        elif model_type == "BERT":
            from transformers import BertTokenizer, BertForSequenceClassification
            self.base_model = BertForSequenceClassification.from_pretrained(
                "bert-base-uncased",early_stopping=False,num_labels=num_classes)
            
        elif model_type == "XLNET":
            from transformers import XLNetTokenizer, XLNetForSequenceClassification
            self.base_model = XLNetForSequenceClassification.from_pretrained(
                "xlnet-base-cased",early_stopping=False,num_labels=num_classes)
            
        elif model_type == "ROBERTA":
            from transformers import RobertaTokenizer, RobertaForSequenceClassification
            self.base_model = RobertaForSequenceClassification.from_pretrained(
                "roberta-base",early_stopping=False,num_labels=num_classes)
        
        elif model_type == "XLMROBERTA":
            from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
            self.base_model = XLMRobertaForSequenceClassification.from_pretrained(
                "xlm-roberta-base", num_labels=num_classes)
        
        elif model_type == "ELECTRA":
            from transformers import ElectraTokenizer, ElectraForSequenceClassification
            self.base_model = ElectraForSequenceClassification.from_pretrained(
                "google/electra-base-discriminator", num_labels=num_classes)
       
        elif model_type == "TAPTBERT":
            from transformers import AutoModel, AutoConfig
            config = AutoConfig.from_pretrained("../input/tapt-v2/config.json")
            self.base_model = AutoModel.from_pretrained("../input/tapt-v2/pytorch_model.bin", config=config)
            self.classifier = nn.Sequential(
                nn.Linear(768, 768), nn.ReLU(), nn.Dropout(p=0.1),
                nn.Linear(768, 768), nn.ReLU(), nn.Dropout(p=0.1),
                nn.Linear(768, num_classes))
    """
        for param in self.base_model.parameters():
            param.requires_grad = True
    """
    
    def forward(self, x):
        ids, mask = x
        
        if self.model_type:
            x = self.base_model(input_ids=ids, attention_mask=mask)
            x = self.classifier(x[1])
            preds = x
        else:
            x = self.base_model(input_ids=ids, attention_mask=mask, labels=None)
            preds = x[0]
        
        preds = F.sigmoid(preds)
        del ids, mask, x
        return preds

In [ ]:
class TabularNet(nn.Module):
    def __init__(self, num_features, hidden_size, dropout, num_classes):
        super(TabularNet, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(dropout*(2/5))
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size*2))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size*2)
        self.dropout2 = nn.Dropout(dropout)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size*2, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(dropout)
        self.layer4 = nn.Linear(hidden_size, num_classes)
        
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.relu(self.dense3(x))
        
        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = self.layer4(x)
        
        x = F.sigmoid(x)
        
        return x

In [ ]:
class MergeNet(nn.Module):
    def __init__(self, num_input, hidden_size, dropout, num_classes):
        super(MergeNet, self).__init__()
        
        
    def forward(self, x):
        return

In [6]:
#define BERT based model
class BertModule(pl.LightningModule):
    def __init__(self, num_features, model_type, num_classes=1):
        super().__init__()
        self.berthead = BertHead(model_type=model_type, num_classes=num_classes)
        self.tabularnet = TabularNet(num_features, hidden_size, dropout, num_classes)
        self.mergenet = MergeNet(num_input, hidden_size, dropout, num_classes)
        
        
    def forward(self, data):
        tab_input, bert_input, target = data
        bert_prob = self.bert_head(bert_input)
        tabular_prob = self.tabularnet(tab_input)
        merge_input = torch.cat([tabular_prob, bert_prob], axis=1)
        preds = self.mergenet(merge_input)
        return preds
    
    
    def training_step(self, batch, batch_idx):
        x, t = batch
        pred = self.forward(x)
        loss = self.criterion(pred, t)
        acc = self.metric(pred, t)
        # you should define log as {"tag_name/log_name"}
        tensorboard_logs = {'train/train_loss': loss, "train/train_acc": acc}
        return {"loss": loss, "acc": acc, "logs": tensorboard_logs, "progress_bar": tensorboard_logs}
    
    
    def validation_step(self, batch, batch_idx):
        x, t = batch
        pred = self.forward(x)
        loss = self.criterion(pred, t)
        acc = self.metric(pred, t)
        logs = {"val_loss": loss, "val_acc": acc}
        return {"val_loss": loss, "val_acc": acc, "progress_bar": logs}

    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([torch.tensor(x['val_acc']) for x in outputs]).mean()
        tensorboard_logs = {'val/avg_loss': avg_loss, "val/avg_acc": avg_acc}
        print(f"val_loss: {avg_loss}, val_acc: {avg_acc}")
        # you should call back as name "val_loss" to using the Early-Stopping
        return {'val_loss': avg_loss, "val_acc": avg_acc, 'log': tensorboard_logs}
    
    
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=1e-5)
        """
        scheduler = {"scheduler": 
                     optimizers.lr_scheduler.CosineAnnealingLR(
                        optimizer, T_max=10),
                    "interval": "epoch",
                    "monitor": "val_loss"}
        """
        return optimizer#[optimizer], [scheduler]
    
    
    def criterion(self, pred, t):
        pred = pred.view(-1)
        pred = pred.float()
        t = t.float()
        return F.binary_cross_entropy(pred,t)
    
    
    def metric(self, pred, t):
        pred = pred.view(-1)
        pred = pred.float()
        t = t.float()
        pred = torch.where(pred<0.5, 0, 1)    
        t, pred = t.to("cpu"), pred.to("cpu").detach().numpy()
        return f1_score(y_true=t, y_pred=pred, average='binary', sample_weight=None, zero_division='warn')

In [7]:
class CFhtmlDataModule(pl.LightningDataModule):
    def __init__(self, csv_file, transform, split_rate, batch_size, num_workers):
        super().__init__()
        self.csv_file = csv_file
        self.transform = transform
        self.split_rate = split_rate
        self.batch_size = batch_size
        self.num_workers = num_workers
        

    def setup(self, stage=None):
        dataset = self.csv_file
        n_samples = len(dataset)
        n_train = int(n_samples * 0.8)
        n_val = n_samples - n_train
        train_dataset, val_dataset = train_test_split(dataset,  train_size=n_train, test_size=n_val)
        
        self.train_dataset = HtmlDataset(csv_file=train_dataset, transform=self.transform)
        self.val_dataset = HtmlDataset(csv_file=val_dataset, transform=self.transform)
        
        
    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                          batch_size=self.batch_size,
                          drop_last=True,
                          num_workers=self.num_workers,
                          pin_memory=True)
    
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset,
                          batch_size=self.batch_size,
                          drop_last=True,
                          num_workers=self.num_workers,
                          pin_memory=True)

In [ ]:
def main():
    # trainer config
    epochs = 400
    output_path = './'
    
    # data module config
    csv_file = train
    model_type = "BERT"
    max_length = 512
    bert_tokenizer = BERT_Tokenize(model_type, max_length)
    transform = bert_tokenizer
    split_rate = 0.8
    batch_size = 16
    num_workers = 4
    
    # model config
    num_classes = 1
    
    # early stopping config
    patience = 5
    
    cf = CFhtmlDataModule(csv_file, transform, split_rate, batch_size, num_workers)
    model = BertModule(model_type, num_classes)
    
    early_stopping = EarlyStopping('val_loss', patience=patience, verbose=True)
    trainer = Trainer(
            max_epochs=epochs,
            weights_save_path=output_path,
            gpus = 1 if torch.cuda.is_available() else None,
            callbacks=[early_stopping]
            #accumulate_grad_batches=1
            # use_amp=False,
        )
        
    trainer.fit(model, cf)
    torch.cuda.empty_cache()
    # TO DO: use model.apply(weights_init) instead of torch.cuda.empty_cache()